In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import numpy as np
import pandas as pd
import sys
import torch as t
from tqdm import tqdm_notebook

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sys.path.append('../numeral_context/')
sys.path.append('../../')

In [3]:
from evaluate_numeral_context import Evaluator
from som.intopolate import weighted_log
from utils.number_handler import to_numeral

In [4]:
from MagNumEvaluator import MagNumEvaluator

In [10]:
numerals = pickle.load(open('data/numerals_mag_1B.pkl','rb'))
print('len of numerals: {}'.format(len(numerals)))

len of numerals: 2342


In [11]:
evaluator = MagNumEvaluator(numerals_dict=numerals, type='MAG')
evaluator.set_varbose(False)

In [12]:
res_mag = {}

In [36]:
pd.DataFrame(res_mag).transpose()

,0,1,2,3
p2,0.384714,0.734415,0.555508,0.998719
p3,0.362938,0.696413,0.547822,0.999146
p5,0.358241,0.675491,0.562767,0.996157
gmm-rd-soft-300,0.523057,0.926132,0.564902,0.993168
gmm-rd-soft-500,0.534159,0.931255,0.559778,0.989325
gmm-rd-hard-300,0.455167,0.771563,0.571734,0.992741
gmm-rd-hard-500,0.478651,0.774552,0.551665,0.989752
gmm-log-rd-soft-300,0.509394,0.970965,0.586251,1.000000
gmm-log-rd-soft-500,0.482921,0.970111,0.611016,1.000000
gmm-log-rd-hard-300,0.589240,0.932963,0.498719,1.000000


In [16]:
print('=== Evaluating Prototype ===')
prototype_size = ['2', '3', '5']
for sz in prototype_size:
    trained_prototypes = pickle.load(
        open('../../data/wikipedia/save/1B30W/prototypes/{}-0005/trained_prototypes_epoch1_{}00_1.0.dat'.format(sz, sz),'rb')) 
    evaluator.load_prototype(trained_prototypes)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['p-{}'.format(sz)] = [OVA, OVAR, SC, BC]

=== Evaluating Prototype ===


In [33]:
print('=== Evaluating Prototype LOG ===')
prototype_size = ['2', '3', '5']
for sz in prototype_size:
    trained_prototypes = pickle.load(
        open('../../data/wikipedia/save/1B30W/prototypes_log/{}-0005/trained_prototypes_epoch1_{}00_1.0.dat'.format(sz, sz),'rb')) 
    evaluator.load_prototype(trained_prototypes)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['p-log-{}'.format(sz)] = [OVA, OVAR, SC, BC]

=== Evaluating Prototype LOG ===


In [19]:
print('=== Evaluating GMM-RD-SOFT ===')
gmms = ['300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/1B30W/gmms/gmm-{}-rd-soft/trained_gmms_epoch1_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm/gmm-{}-rd-soft.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['gmm-rd-soft-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-SOFT ===


In [26]:
print('=== Evaluating GMM-RD-SOFT-LOG ===')
gmms = ['300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/1B30W/gmms_log/gmm-{}-rd-soft/trained_gmms_epoch1_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm_log/gmm-{}-rd-soft.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, log_space=True)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['gmm-log-rd-soft-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-SOFT-LOG ===


In [21]:
print('=== Evaluating GMM-RD-HARD ===')
gmms = ['300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/1B30W/gmms/gmm-{}-rd-hard/trained_gmms_epoch1_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm/gmm-{}-rd-hard.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm,)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['gmm-rd-hard-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-HARD ===


In [27]:
print('=== Evaluating GMM-RD-HARD-LOG ===')
gmms = ['300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/1B30W/gmms_log/gmm-{}-rd-hard/trained_gmms_epoch1_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm_log/gmm-{}-rd-hard.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, log_space=True)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_mag['gmm-log-rd-hard-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-HARD-LOG ===


In [30]:
print('=== Evaluating LSTM Baseline ===')
idx2vec_o = pickle.load(open('../../data/wikipedia/save/1B30W/LSTM/idx2vec_o_epoch1.dat','rb'))
idx2vec_i = pickle.load(open('../../data/wikipedia/save/1B30W/LSTM/idx2vec_i_epoch1.dat','rb'))
LSTM_model_path = '../../data/wikipedia/save/1B30W/LSTM/sgns_epoch1.pt'
evaluator.load_LSTM(idx2vec_i, idx2vec_o, LSTM_model_path)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_mag['LSTM'] = [OVA, OVAR, SC, BC]

=== Evaluating LSTM Baseline ===


In [32]:
print('=== Evaluating Token Baseline ===')
idx2word = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsTokenUnkNumeral30W/idx2word.dat','rb'))
word2idx = pickle.load(open('../../data/wikipedia/preprocess1B/NumeralAsTokenUnkNumeral30W/word2idx.dat','rb'))
evaluator.reload(numerals_dict=numerals, idx2word=idx2word, word2idx=word2idx)

idx2vec_o = pickle.load(open('../../data/wikipedia/save/1B30W/token/idx2vec_o_epoch1.dat','rb'))
idx2vec_i = pickle.load(open('../../data/wikipedia/save/1B30W/token/idx2vec_i_epoch1.dat','rb'))
evaluator.load_TOKEN(idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_mag['TOKEN'] = [OVA, OVAR, SC, BC]

=== Evaluating Token Baseline ===


In [35]:
print('=== Evaluating Fixed Baseline ===')
evaluator.load_fixed(idx2vec_i=idx2vec_i)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_mag['FIXED'] = [OVA, OVAR, SC, BC]

=== Evaluating Fixed Baseline ===
